<a href="https://colab.research.google.com/github/kavyajeetbora/recipe_recommender/blob/master/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from glob import glob
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import streamlit as st

In [65]:
files = glob(r"data\*.parquet")
df = pd.read_parquet(files)
df['steps'] = df['steps'].str.strip("[]")
df.iloc[0]['steps']

"'make a choice and proceed with recipe', 'depending on size of squash , cut into half or fourths', 'remove seeds', 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece', 'season with mexican seasoning mix ii', 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece', 'season with sweet mexican spice mix', 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin', 'be careful not to burn the squash especially if you opt to use sugar or butter', 'if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking', 'if desired , season with salt'"

In [66]:
files = glob(r"data\*.parquet")
df = pd.read_parquet(files)
df['ingredients'] = df['ingredients'].str.strip("[]").str.replace("'","").str.replace('"',"").str.split("',").apply(lambda x: [y.strip() for y in x])
df['steps'] = df['steps'].str.strip("[]").str.replace('"',"").str.split("',").apply(lambda x: [y.replace("'","").strip() for y in x])
print("Shape of the dataframe",df.shape)
df.head(3)

Shape of the dataframe (231637, 15)


,name,minutes,n_steps,steps,description,ingredients,n_ingredients,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),embedding
0,arriba baked winter squash mexican style,55,11,"['make a choice and proceed with recipe, 'depe...",autumn is my favorite time of year to cook! th...,"[winter squash, mexican seasoning, mixed spice...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0,"[-0.039017964, 0.030269215, 0.010577081, 0.089..."
1,a bit different breakfast pizza,30,9,"['preheat oven to 425 degrees f, 'press dough ...",this recipe calls for the crust to be prebaked...,"[prepared pizza crust, sausage patty, eggs, mi...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0,"[-0.06494449, 0.008968874, 0.059068605, 0.0291..."
2,all in the kitchen chili,130,6,"['brown ground beef in large pot, 'add chopped...",this modified version of 'mom's' chili was a h...,"[ground beef, yellow onions, diced tomatoes, t...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0,"[-0.10917001, -0.01785316, 0.0396873, 0.044659..."


In [3]:
def cosine_similarity(vec1,vec2):
    '''
    Returns the cosine similarity between two vectors of n dimension
    '''
    denom = np.sqrt(np.sum(np.square(vec1))) * np.sqrt(np.sum(np.square(vec2)))
    return np.round(np.dot(vec1,vec2) / denom * 100, 2)

In [4]:
index = 999
data = df.iloc[index]
recipe, vector = data['name'], data['embedding']

print("Name of the dish:", recipe)

Name of the dish: 1890 cream cake


In [5]:
%%time

df_result = df.copy()
df_result['similarity'] = df_result['embedding'].apply(lambda x : cosine_similarity(vector, x))
df_result.sort_values(by="similarity", ascending=False).iloc[1:4]

CPU times: total: 4.08 s
Wall time: 4.08 s


,name,minutes,n_steps,steps,description,ingredients,n_ingredients,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),embedding,similarity
136066,south african beesting cake with custard filling,105,24,['preheat oven to 350 deg f / 180 deg celsius'...,"i do not know where the ""bee-sting"" comes from...","['flour', 'baking powder', 'salt', 'butter', '...",13,500.9,49.0,83.0,15.0,16.0,83.0,15.0,"[-0.049224626, -0.03398056, 0.03450093, 0.0180...",95.68
170691,white sponge cake,70,21,"['preheat oven to 400 degrees f', 'grease 3- 1...",nice light white cake. delicious served filled...,"['egg yolk', 'eggs', 'sugar', 'salt', 'baking ...",10,584.2,30.0,168.0,26.0,24.0,13.0,29.0,"[-0.035752527, 0.016094102, 0.029773716, -0.01...",95.26
22010,frankfurter kranz buttercream filled cake,90,27,"['preheat oven to 350f', 'cream butter with su...",this recipe is posted by request-please read t...,"['butter', 'granulated sugar', 'eggs', 'lemon,...",11,5544.0,579.0,1434.0,137.0,156.0,925.0,165.0,"[-0.041397896, -0.0044432464, 0.03030667, 0.00...",95.19


In [6]:
x = df_result.sample(10)
x.head(2)

,name,minutes,n_steps,steps,description,ingredients,n_ingredients,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),embedding,similarity
21665,foolproof sourdough starter,4330,15,['mix the milk and yogurt together in a glass ...,this was found at alaska.net and trust me work...,"['plain yogurt', 'milk', 'unbleached flour']",3,380.3,13.0,23.0,4.0,29.0,27.0,19.0,"[-0.020224523, -0.011550143, 0.045514733, 0.03...",78.72
221425,dari s white chicken chili,40,6,['heat the olive oil in a large skillet or sau...,chili with a twist. nothing to warm you up be...,"['olive oil', 'onions', 'garlic cloves', 'drie...",15,533.5,33.0,14.0,30.0,80.0,50.0,12.0,"[-0.091111556, 0.0023644804, 0.0027534447, 0.0...",64.99


21665                [plain yogurt, milk, unbleached flour]
221425    [olive oil, onions, garlic cloves, dried orega...
29766     [active dry yeast, sugar, warm water, salt, wh...
111908    [jumbo shrimp, all-purpose flour, panko breadc...
132444    [red potatoes, extra virgin olive oil, sour cr...
39816     [pumpkin puree, peanut butter, water, orange j...
7894      [egg, applesauce, butter, dark brown sugar, gr...
125054    [cornbread stuffing mix, celery, onion, butter...
121587    [peaches, bananas, maraschino cherry, lemon ju...
98461                                 [flour, butter, milk]
Name: ingredients, dtype: object

In [29]:
min = x.iloc[0, 1]
st.metric(label='Minutes', value=min)

2024-01-11 16:44:22.159 
  command:

    streamlit run c:\Users\kbora\Anaconda3\envs\recipe-env\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [26]:
y = x.iloc[0, 8:13]
fig = go.Figure(
    go.Bar(
        x = list(y.index),
        y = list(y.values),
        width = 0.2
    )
)
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    bargap=1,
    bargroupgap=0.0
)
fig.update_xaxes(
    showgrid=False,
)
fig.update_yaxes(
    showgrid=False,
    showticklabels=False
)
fig.show()

In [8]:

fig = px.bar(y, x='year', y='pop',
             hover_data=['lifeExp', 'gdpPercap'], color='lifeExp',
             labels={'pop':'population of Canada'}, height=400)
fig.show()

NameError: name 'px' is not defined